# Instalando Dependencias

In [ ]:
%pip install pandas

# Schema para o DB

# Criando um DB

In [ ]:
import sqlite3
from sqlite3 import Connection

class DbConnectionHandler:
    def __init__(self) -> None:
        self.__connection_string = "storage.db"
        self.__conn = None  

    def connect(self) -> None:
        conn = sqlite3.connect(self.__connection_string, check_same_thread=False)

        with open(r'schema/schema.sql', 'r') as f:
            conn.executescript(f.read())

        self.__conn = conn

    def get_connection(self) -> Connection:
        return self.__conn

db_conn_handler = DbConnectionHandler()

In [ ]:
db_conn_handler.connect()

# Lendo as Tabelas

In [ ]:
import pandas as pd

tabela_transacoes = pd.read_csv("data/Tabela_de_Transacoes.csv", sep=";")
tabela_eventos = pd.read_csv("data/Tabela_de_Eventos.csv", sep=";")

# Inserindo para o DB

In [ ]:
tabela_transacoes.to_sql("Transacoes", db_conn_handler.get_connection(), if_exists="append",index=False)

In [ ]:
tabela_eventos.to_sql("Eventos", db_conn_handler.get_connection(), if_exists="append",index=False)

# Extraindo Indicadores

In [ ]:
class IndicadoresMarketing:
    def __init__(self) -> None:
        self.conn = db_conn_handler.get_connection()
        self.table = "Eventos"
    
    def calcular_indicadores_geral(self):
        cursor = self.conn.cursor()

        query = f'''
        SELECT 
            SUM(impressoes) AS total_impressoes,
            SUM(cliques) AS total_cliques,
            SUM(visualizacoes) AS total_visualizacoes,
            SUM(nova_conversao) AS total_conversoes,
            SUM(custo) AS total_custo,
            SUM(receita_gerada) AS total_receita
        FROM {self.table}
        '''
        cursor.execute(query)
        row = cursor.fetchone()

        total_impressoes, total_cliques, total_visualizacoes, total_conversoes, total_custo, total_receita = row

        ctr = ((total_cliques / total_impressoes) * 100) if total_impressoes else 0
        cpc = (total_custo / total_cliques) if total_cliques else 0
        cpv = (total_custo / total_visualizacoes) if total_visualizacoes else 0
        cac = (total_custo / total_conversoes) if total_conversoes else 0
        roi = ((total_receita - total_custo) / total_custo) if total_custo else 0

        return {
            "CTR": round(ctr, 4),
            "CPC": round(cpc, 2),
            "CPV": round(cpv, 2),
            "CAC": round(cac, 4),
            "ROI": round(roi, 2)
        }
    
    def calcular_indicadores_por_campanha(self, client_id):
        cursor = self.conn.cursor()
        query = f'''
        SELECT 
            impressoes,
            cliques,
            visualizacoes,
            nova_conversao,
            custo,
            receita_gerada
        FROM {self.table}
        WHERE campanha = ?
        '''
        cursor.execute(query, (client_id,))
        
        row = cursor.fetchone()

        if row is None:
            return f"Nenhum registro encontrado para id={client_id}"

        impressoes, cliques, visualizacoes, nova_conversao, custo, receita_gerada = row

        ctr = ((cliques / impressoes) * 100) if impressoes else 0
        cpc = (custo / cliques) if cliques else 0
        cpv = (custo / visualizacoes) if visualizacoes else 0
        cac = (custo / nova_conversao) if nova_conversao else 0
        roi = ((receita_gerada - custo) / custo) if custo else 0

        return {
            "CTR": round(ctr, 4),
            "CPC": round(cpc, 2),
            "CPV": round(cpv, 2),
            "CAC": round(cac, 4),
            "ROI": round(roi, 2)
        }
    
indicadores_marketing = IndicadoresMarketing()

In [ ]:
indicadores_geral = indicadores_marketing.calcular_indicadores_geral()

In [ ]:
for indicador, valor  in indicadores_geral.items():
    print(f"{indicador}: {valor}")

In [ ]:
indicadores_campanha = indicadores_marketing.calcular_indicadores_por_campanha("SkinCareVerão")

In [ ]:
for indicador, valor in indicadores_campanha.items():
    print(f"{indicador}: {valor}")